# Data Cleaning — Goodbooks-10k Top 100 (Updated)
**Objetivo:**  
 
1. Cargar y explorar los ficheros de ratings, metadatos y tags.  
2. Seleccionar los 100 libros más valorados.  
3. Filtrar usuarios con actividad mínima (≥ 20 ratings).  
4. Binarizar las calificaciones (≥ 4 → “like”).  
5. Pivotar a matriz dispersa 0/1 `R_binary`.  
6. Extraer y guardar metadata enriquecida (título, autor, top-tags).



Importamos las librerías

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

Cargamos los datos

In [3]:
ratings = pd.read_csv('./data/ratings.csv')
books = pd.read_csv('./data/books.csv')

Limpieza: eliminamos usuarios y libros con menos de 5 valoraciones

In [4]:
min_ratings = 5
user_counts = ratings['user_id'].value_counts()
book_counts = ratings['book_id'].value_counts()

ratings_clean = ratings[
    ratings['user_id'].isin(user_counts[user_counts >= min_ratings].index) &
    ratings['book_id'].isin(book_counts[book_counts >= min_ratings].index)
].copy()

n_users_clean = ratings_clean['user_id'].nunique()
n_items_clean = ratings_clean['book_id'].nunique()

print(f"Usuarios con ≥{min_ratings} ratings: {n_users_clean}")
print(f"Libros   con ≥{min_ratings} ratings: {n_items_clean}")

Usuarios con ≥5 ratings: 53424
Libros   con ≥5 ratings: 10000


Dividimos en train y test, tomando valoración y usuario

In [5]:
train, test = train_test_split(ratings_clean, test_size=0.2, random_state=42)

Baseline: media global

In [6]:
global_mean = train['rating'].mean()
test['pred_global_mean'] = global_mean

# mean squared error y mean absolute error
rmse_global = np.sqrt(mean_squared_error(test['rating'], test['pred_global_mean']))
mae_global = mean_absolute_error(test['rating'], test['pred_global_mean'])

Baseline: media por usuario

In [7]:
user_mean = train.groupby('user_id')['rating'].mean()
test = test.join(user_mean, on='user_id', rsuffix='_user_mean')
test['pred_user_mean'] = test['rating_user_mean'].fillna(global_mean)

rmse_user = np.sqrt(mean_squared_error(test['rating'], test['pred_user_mean']))
mae_user = mean_absolute_error(test['rating'], test['pred_user_mean'])

Mostramos los resultados

In [8]:
results = pd.DataFrame({
    'Baseline': ['Global Mean', 'User Mean'],
    'RMSE': [rmse_global, rmse_user],
    'MAE': [mae_global, mae_user]
})

print('Resultados de Baseline\n', results)

Resultados de Baseline
       Baseline      RMSE       MAE
0  Global Mean  0.990329  0.774018
1    User Mean  0.893159  0.700713


RMSE (~1): en promedio, la predicción se equivoca en 1 estrella sobre 5.
MAE (~0.7): el error medio absoluto es  de, más o menos, 0.7 estrellas.

Al pasar de global mean a user mean el RMSE se reduce a 0.89, por lo que ya tenemos una mejora del 10%. Estos datos serán el baseline contra el que compararemos los 3 métodos de filtrado colaborativo para ver si nos son útiles.



Para comenzar con las implementaciones guardaremos en nuestra carpeta data/processed ratings_clean, train y test para mayor orden.

In [9]:
import os
processed_dir = 'data/processed'

if not os.path.isdir(processed_dir):
    os.makedirs(processed_dir, exist_ok=True)
    
ratings_clean.to_csv(os.path.join(processed_dir, 'ratings_clean.csv'), index=False)
train.to_csv(os.path.join(processed_dir, 'train.csv'), index=False)
test.to_csv(os.path.join(processed_dir, 'test.csv'), index=False)

print("Datos guardados en 'data/processed':")
print(os.listdir(processed_dir))

Datos guardados en 'data/processed':
['ratings_clean.csv', 'test.csv', 'train.csv']


In [1]:
def get_user_ndcg(u, pred_matrix, test_df, K=5, rel_col='rel'):
    """
    nDCG@K para el usuario u (0-based).
    test_df debe tener columnas ['user_id','book_id', rel_col].
    rel_col es la relevancia (binaria o graduada).
    """
    user_ratings = test_df[test_df.user_id == (u+1)]
    if user_ratings.empty:
        return None

    true_rels = {
        int(row.book_id)-1: row[rel_col]
        for _, row in user_ratings.iterrows()
    }

    scores = pred_matrix[u]
    # Top-K 
    top_k = np.argsort(scores)[::-1][:K]

    dcg = 0.0
    for rank, item in enumerate(top_k, start=1):
        rel = true_rels.get(item, 0)
        dcg += (2**rel - 1) / np.log2(rank + 1)

    ideal_rels = sorted(true_rels.values(), reverse=True)[:K]
    idcg = sum((2**rel - 1) / np.log2(idx + 1)
               for idx, rel in enumerate(ideal_rels, start=1))

    return dcg / idcg if idcg > 0 else 0.0

def get_ndcg(pred_matrix, test_df, K=5, rel_col='rel'):
    """
    nDCG@K promedio sobre todos los usuarios con al menos una valoración.
    """
    total, count = 0.0, 0
    num_users = pred_matrix.shape[0]
    for u in range(num_users):
        val = get_user_ndcg(u, pred_matrix, test_df, K, rel_col)
        if val is not None:
            total += val
            count += 1
    return (total / count) if count > 0 else 0.0


#### Baseline: User-Mean Predictor

1. **¿Qué hace?** Para cada cada usuario siempre su **media histórica** de ratings (o la media global si es nuevo).  
2. **Ventaja**: captura el sesgo individual de cada lector (alguien exigente vs. alguien generoso).  
3. **Regresión**: evalúa MAE/RMSE comparando la media de usuario vs. rating real.  
4. **Clasificación**: binariza con umbral = floor(media global) para medir Precision, Recall y F1.  
5. **Ranking**: usa la misma matriz de predicciones para calcular nDCG@10, mostrando su capacidad de ordenar ítems.  

In [ ]:
import numpy as np
import pandas as pd
from math import floor
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error,
    precision_score, recall_score, f1_score
)

train_df = pd.read_csv('./data/processed/train.csv')
test_df  = pd.read_csv('./data/processed/test.csv')

# medias
global_mean = train_df.rating.mean()
item_mean   = train_df.groupby('book_id').rating.mean().to_dict()

# item-mean como predicción
NUM_USERS = int(max(train_df.user_id.max(), test_df.user_id.max()))
NUM_ITEMS = int(max(train_df.book_id.max(), test_df.book_id.max()))
pred_matrix = np.zeros((NUM_USERS, NUM_ITEMS), dtype=np.float32)

for uid in range(1, NUM_USERS+1):
    for iid in range(1, NUM_ITEMS+1):
        mu_i = item_mean.get(iid, global_mean)
        pred_matrix[uid-1, iid-1] = mu_i

# regresión en test
y_true, y_pred = [], []
for _, r in test_df.iterrows():
    u, i = int(r.user_id)-1, int(r.book_id)-1
    y_true.append(r.rating)
    y_pred.append(pred_matrix[u, i])

mae  = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))

# binarizamos y métricas de clasificación
thr = floor(global_mean)
y_true_bin = (np.array(y_true) >= thr).astype(int)
y_pred_bin = (np.array(y_pred) >= thr).astype(int)

prec = precision_score(y_true_bin, y_pred_bin, zero_division=0)
rec  = recall_score   (y_true_bin, y_pred_bin, zero_division=0)
f1   = f1_score       (y_true_bin, y_pred_bin, zero_division=0)

# ranking uniforme
ndcg10 = 0.0

# imprimimos
print(f"Item-Mean Baseline → MAE: {mae:.4f}, RMSE: {rmse:.4f}")
print(f"Item-Mean Baseline → Precision: {prec:.3f}, Recall: {rec:.3f}, F1: {f1:.3f}")
print(f"Item-Mean Baseline → nDCG@10: {ndcg10:.3f}")

